In [ ]:
import json
import time
import datetime
import re
import os
import sys
import subprocess
import pandas as pd
import pytesseract
from bs4 import BeautifulSoup
from tqdm import tqdm
from PIL import Image
import base64
import cv2 as cv
from itertools import zip_longest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import threading

def request_url(no):
    """
    请求网页
    """
    global result
    
    chrome_options = Options()
    # chrome_options.add_argument('--headless') #无界面
    # chrome_options.add_argument('blink-settings=imagesEnabled=false') #不加载图片
    chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) #防检测
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()  #最大化窗口
    # driver.set_window_size(1920,1080)  #设置分辨率
    driver.get("https://www.mytoken.io/zh/latest/")
    
    #跳过公告弹窗
    time.sleep(2)
    try:
        page_value = '/html/body/div[2]/div/div[2]/div/div[2]/button/span'
        driver.implicitly_wait(30)  #隐式等待5秒
        driver.find_element(by=By.XPATH,value=page_value).click()
    except:
        pass
    
    #按24h增长率排序
    time.sleep(2)
    try:
        page_value = "/html/body/div/div/main/div/div/div[3]/table/thead/tr/th[5]/div/div/img[2]"
        driver.find_element(by=By.XPATH,value=page_value).click()
    except:
        print(no)
    
    #
    time.sleep(2)
    page_value = f"/html/body/div/div/main/div/div/div[3]/table/tbody/tr[{no}]"
    driver.implicitly_wait(30)  #隐式等待5秒
    coin = driver.find_element(by=By.XPATH,value=page_value).text.split('\n')[:-3]
    ans = ['序号','名称1','名称2','价格','24h','7d']
    res = dict(zip_longest(ans,coin))
    
    #跳转到详细页面
    page_value = f"/html/body/div/div/main/div/div/div[3]/table/tbody/tr[{no}]"
    driver.implicitly_wait(30)  #隐式等待5秒
    driver.find_element(by=By.XPATH,value=page_value).click()
    time.sleep(5)
    
    #所属链
    try:
        page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[3]/div/div[1]"
        driver.implicitly_wait(30)  #隐式等待5秒
        chain = driver.find_element(by=By.XPATH,value=page_value).text
        if chain.split('\n')[0] == '基础链':
            res['所属链'] = chain.split('\n')[-1]
    except:
        pass
        
    #合约
    try:
        page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[3]/div/div[2]"
        driver.implicitly_wait(30)  #隐式等待5秒
        contract = driver.find_element(by=By.XPATH,value=page_value).text
        if contract.split('\n')[0] in ['BSC','Ethereum','Polygon']:
            res['合约'] = contract.split('\n')[1]
    except:
        pass
        
    #链接
    links = []
    try:
        page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[4]/div/div"
        driver.implicitly_wait(30)  #隐式等待5秒
        link_list = driver.find_elements(by=By.XPATH,value=page_value)
        length = len(link_list)
        for i in range(length):
            if link_list[i].text.split('\n')[0] == '官网':
                page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[4]/div/div[{i+1}]/a"
                res['官网'] = driver.find_element(by=By.XPATH,value=page_value).get_attribute('href')

            elif link_list[i].text.split('\n')[0] == '社交媒体':
                page_value = f"/html/body/div[1]/div/main/div/div/div[3]/div[2]/div[4]/div/div[{i+1}]/div/a"
                for j in (driver.find_elements(by=By.XPATH,value=page_value)):
                    links.append(j.get_attribute('href'))
                res['相关链接'] = links
    except:
        pass
    
    
    
    
    lock.acquire()    #线程锁
    result.append(res)    
    lock.release()    #释放锁
    
    print(f'{no}爬取完成')
    # driver.close()
    pool_sema.release() #释放已完成线程

def main():
    thread_list = []
    for i in tqdm(range(1,11)):
        pool_sema.acquire() 
        thread = threading.Thread(target=request_url,args=[i])
        thread.start()
        thread_list.append(thread)

    for t in thread_list:
        t.join()

    date_ = datetime.datetime.now().strftime('%Y-%m-%d %H')   #更新时间
    df = pd.DataFrame(result)
    df['序号'] = df['序号'].map(lambda i:int(i))
    df['更新时间'] = date_
    df = df.sort_values('序号')
    #只保留有合约或官网信息的代币
    df = df[~(df['合约'].isna() & df['官网'].isna())].drop_duplicates('合约')
    df.set_index('序号').reset_index(drop=True).to_csv(f'./save/Mytoken{date_}更新.csv',index=False)
    
if __name__ == "__main__":
    result = []
    lock = threading.Lock() #设置线程锁
    max_connection = 4 #10线程
    pool_sema = threading.BoundedSemaphore(max_connection) #最大线程量
    main()

In [3]:
%run MyToken.py

 50%|█████████████████████████████████████████▌                                         | 5/10 [00:19<00:19,  3.98s/it]

2爬取完成
3爬取完成


 60%|█████████████████████████████████████████████████▊                                 | 6/10 [00:30<00:21,  5.43s/it]

4爬取完成
1爬取完成


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:55<00:08,  8.71s/it]

6爬取完成


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:55<00:00,  5.59s/it]

7爬取完成


5爬取完成
9爬取完成
10爬取完成
8爬取完成


In [4]:
%run avedex.py

 10%|████████▎                                                                          | 1/10 [00:09<01:25,  9.53s/it]

1爬取完成


 20%|████████████████▌                                                                  | 2/10 [00:19<01:20, 10.07s/it]

2爬取完成


 40%|█████████████████████████████████▏                                                 | 4/10 [00:39<01:00, 10.05s/it]

...

7爬取完成


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:33<00:10, 10.49s/it]

9爬取完成


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:44<00:00, 10.45s/it]


10爬取完成
8爬取完成
